# SYMBOLIC STUDY
## **SPATIAL SLIDER CRANK**
----------------

### SYSTEM DISCRIPTION
---------------------

_Double click to write a discription here ..._

-----------------------------------------------

In [1]:
import sys
pkg_path = '/home/khaledghobashy/Documents/asurt_cdt_symbolic'
sys.path.append(pkg_path)

In [2]:
project_dir = '/home/khaledghobashy/Documents/asurt_cdt_projects/asurt_projects'

In [3]:
import asurt.interfaces.scripting as sui

## SYSTEM TOPOLOGY
-----------------

In [4]:
model = sui.standalone_topology('spatial_slider_crank')

### ADDING SYSTEM BODIES

In [5]:
model.add_body('crank')
model.add_body('conct')
model.add_body('slider')

### ADDING SYSTEM JOINTS

In [6]:
model.add_joint.revolute('a', 'rbs_crank', 'ground')
model.add_joint.spherical('b', 'rbs_crank', 'rbs_conct')
model.add_joint.universal('c', 'rbs_conct', 'rbs_slider')
model.add_joint.translational('d', 'rbs_slider', 'ground')

### ADDING SYSTEM ACTUATORS

In [7]:
model.add_actuator.rotational_actuator('act', 'jcs_a')

### ADDING SYSTEM FORCES

In [ ]:
#model.add_force.internal_force('force_name', 'body_1', 'body_2', mirror=False)

### ASSEMBLING AND SAVING SYSTEM

In [8]:
model.assemble_model()
model.save()

In [9]:
model.write_python_code(project_dir)

File full path : /home/khaledghobashy/Documents/asurt_cdt_projects/asurt_projects/numenv/python/models/standalones/spatial_slider_crank.py


In [11]:
model._mbs.pos_equations

Matrix([
[                                                                                                           -{R_{ground}} + A({P_{rbs_crank}})*{\bar{u}^{rbs_crank}_{jcs_a}} - A({P_{ground}})*{\bar{u}^{ground}_{jcs_a}} + {R_{rbs_crank}}],
[                                                                                                                                         Mbar_rbs_crank_jcs_a[:,(0, 1)].T*A({P_{rbs_crank}}).T*A({P_{ground}})*Mbar_ground_jcs_a[:,(2, 3)]],
[                                                                                                                                         Mbar_rbs_crank_jcs_a[:,(1, 2)].T*A({P_{rbs_crank}}).T*A({P_{ground}})*Mbar_ground_jcs_a[:,(2, 3)]],
[(-sin(AF_jcs_a(t)))*Mbar_rbs_crank_jcs_a[:,(0, 1)].T*A({P_{rbs_crank}}).T*A({P_{ground}})*Mbar_ground_jcs_a[:,(0, 1)] + cos(AF_jcs_a(t))*Mbar_rbs_crank_jcs_a[:,(1, 2)].T*A({P_{rbs_crank}}).T*A({P_{ground}})*Mbar_ground_jcs_a[:,(0, 1)]],
[                                      

## SYSTEM CONFIGURATION
-----------------------

In [12]:
config_name = 'spatial_slider_crank_cfg'
config = sui.configuration(config_name, model)

### CONFIGURATION INPUTS

In [13]:
config.add_point.UserInput('a')
config.add_point.UserInput('b')
config.add_point.UserInput('c')
config.add_point.UserInput('d')

config.add_vector.UserInput('v')

### CONFIGURATION RELATIONS

In [14]:
config.add_relation.Equal_to('pt1_jcs_a', ('hps_a',))

config.add_relation.Equal_to('pt1_jcs_b', ('hps_b',))

config.add_relation.Equal_to('pt1_jcs_c', ('hps_c',))
config.add_relation.Oriented('ax1_jcs_c', ('hps_c', 'hps_b'))
config.add_relation.Equal_to('ax2_jcs_c', ('vcs_v',))

config.add_relation.Equal_to('pt1_jcs_d', ('hps_d',))
config.add_relation.Equal_to('ax1_jcs_d', ('vcs_v',))

### CONFIGURATION GEOMETRIES

In [15]:
config.add_scalar.UserInput('crank_raduis')
config.add_scalar.UserInput('conct_raduis')
config.add_scalar.UserInput('slider_raduis')

In [16]:
config.add_point.UserInput('s1')
config.add_point.UserInput('s2')

In [17]:
config.add_geometry.Cylinder_Geometry('crank', ('hps_a', 'hps_b', 's_crank_raduis'))
config.assign_geometry_to_body('rbs_crank', 'gms_crank')

config.add_geometry.Cylinder_Geometry('conct', ('hps_b', 'hps_c', 's_conct_raduis'))
config.assign_geometry_to_body('rbs_conct', 'gms_conct')

config.add_geometry.Cylinder_Geometry('slider', ('hps_s1', 'hps_s2', 's_slider_raduis'))
config.assign_geometry_to_body('rbs_slider', 'gms_slider')

### ASSEMBLING AND SAVING CONFIGURATION

In [18]:
config.assemble_model()
config.extract_inputs_to_csv()

In [19]:
config.write_python_code(project_dir)

File full path : /home/khaledghobashy/Documents/asurt_cdt_projects/asurt_projects/numenv/python/models/configurations/spatial_slider_crank_cfg.py


In [20]:
config.write_blender_script(project_dir)